In [7]:
import sys
sys.path.append("../") 

import numpy as np
from tools import signature as sig

from ipywidgets import interact, fixed
import ipywidgets as widgets
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from ipywidgets import Image, Layout
import PIL.Image
import io

In [8]:
jsonfilename='../json/signature_all_100.json'

In [9]:
# Build colorscale
dict_data = sig.loadJson(jsonfilename)

x = np.array(dict_data['x'])
labels = dict_data['labels']
predictions = dict_data['predictions']
hidden_layers = [np.array(e) for e in dict_data['hidden_layers']]
clusters = dict_data['clusters']

keep = dict_data['keep']
best_k = dict_data['best_k']

colorscale = {
    'color': np.zeros(len(x), dtype='uint8'),
    'colorscale': ['grey', '#ee1717', '#7201a8', '#17a60d', '#291ae0', '#f418ff', '#e3ae04', 
                    '#fb5e09', 'black', '#0ed6e1', '#d35400'],
    'cmin': 0,
    'cmax': 10
}

traces = sig.umap_plot(x, labels, predictions, hidden_layers, clusters, colorscale)

parcats = sig.parcats(labels, keep, best_k, clusters, predictions, colorscale)

traces.append(parcats)

FileNotFoundError: [Errno 2] No such file or directory: '../json/signature_all_100.json'

In [4]:
def image_to_byte_array(image):
    imgByteArr = io.BytesIO()
    image.save(imgByteArr, format='png')
    imgByteArr = imgByteArr.getvalue()
    return imgByteArr

def to_image(x):
    return  image_to_byte_array(
                PIL.Image.fromarray(
                    (x.reshape(28 , 28) * 255).astype(np.uint8)
                ))

# Update color callback
def update_color(trace, points, state):
    # Compute new color array
    new_color = np.array(trace.marker.color)
    new_color[points.point_inds] = color_toggle.index
    with fig.batch_update():
        # Update scatter color
        for i in range(len(fig.data)-1):
            fig.data[i].marker.color = new_color

        # Update parcats colors
        fig.data[len(fig.data)-1].line.color = new_color
        
# Update color callback
def update_parcats_color(trace, points, state):
    # Compute new color array
    new_color = np.array(trace.line.color)
    new_color[points.point_inds] = color_toggle.index
    with fig.batch_update():
        # Update scatter color
        for i in range(len(fig.data)-1):
            fig.data[i].marker.color = new_color

        # Update parcats colors
        fig.data[len(fig.data)-1].line.color = new_color
        
def print_image(trace, points, state):     
    # Image update
    if (points.point_inds != []):
        image_widget.value = to_image(x[points.point_inds[0]])

In [5]:
fig = make_subplots(rows=3, figure=go.FigureWidget(), specs=[[{'type': 'xy'}],
                                                             [{'type': 'xy'}],
                                                            [{'type': 'parcats'}]])

fig.add_traces(traces, rows=[1, 2, 3], cols=[1, 1, 1])

fig.update_layout(height=800,
                    dragmode='lasso', hovermode='closest')

# Build color selection widget
color_toggle = widgets.ToggleButtons(
    options=['None', 'Red', 'Purple', 'Green', 'Blue', 'Pink', 
                 'Yellow', 'Orange','Black' ,'Aqua', 'Brown'],
    index=1, description='Color:', disabled=False)
    
image_widget = Image(
    value=to_image(x[0]),
    layout=Layout(height='100px', width='100px')
)

# Register callback on scatter selection...
fig.data[0].on_selection(update_color)
fig.data[1].on_selection(update_color)

fig.data[0].on_hover(print_image)
fig.data[1].on_hover(print_image)

# and parcats click
fig.data[2].on_click(update_parcats_color)

# Display figure
widgets.VBox([color_toggle, image_widget, fig])

NameError: name 'traces' is not defined